In [1]:
import pandas as pd
import os
import numpy as np
import itertools
import xgboost as xgb
from scipy.spatial.distance import euclidean
import joblib
import datetime

pd.set_option('display.max_columns', None)

# Load Data

In [ ]:
dsdir = 'dataset/coupon-purchase-prediction'

In [ ]:
train = pd.read_csv('CPP_REPRO_cl_train.csv').sample(frac=1, random_state=0).reset_index(drop=True)
test =  pd.read_csv('CPP_REPRO_cl_test.csv')
submission = pd.read_csv(os.path.join(dsdir,'sample_submission.csv'))

# Preprocess Some Columns

In [ ]:
train.DISPFROM = pd.to_datetime(train.DISPFROM)
train.DISPEND = pd.to_datetime(train.DISPEND)
train.VALIDFROM = pd.to_datetime(train.VALIDFROM)
train.VALIDEND = pd.to_datetime(train.VALIDEND)
train.REG_DATE = pd.to_datetime(train.REG_DATE)
train.WITHDRAW_DATE = pd.to_datetime(train.WITHDRAW_DATE)

test.DISPFROM = pd.to_datetime(test.DISPFROM)
test.DISPEND = pd.to_datetime(test.DISPEND)
test.VALIDFROM = pd.to_datetime(test.VALIDFROM)
test.VALIDEND = pd.to_datetime(test.VALIDEND)
test.REG_DATE = pd.to_datetime(test.REG_DATE)
test.WITHDRAW_DATE = pd.to_datetime(test.WITHDRAW_DATE)

In [ ]:
train.drop(['DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)
test.drop(['DISPFROM', 'DISPEND', 'VALIDFROM', 'VALIDEND', 'REG_DATE', 'WITHDRAW_DATE'], axis=1, inplace=True)

In [ ]:
#train.drop(['USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY'], axis=1, inplace=True)
#test.drop(['USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY'], axis=1, inplace=True)

In [ ]:
train['SAME_PREF'] = train['PREF_NAME_COUPON'] == train['PREF_NAME_USER']
test['SAME_PREF'] =  test['PREF_NAME_COUPON'] == test['PREF_NAME_USER']

In [ ]:
def calc_euc(row):
    return euclidean((row['LATITUDE_COUPON'],row['LONGITUDE_COUPON']),(row['LATITUDE_USER'],row['LONGITUDE_USER']))

In [ ]:
print('TRAIN-LONGITUDE_DIST')
train['LONGITUDE_DIST'] = train.apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)
print('TRAIN-LATITUDE_DIST')
train['LATITUDE_DIST'] = train.apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)
print('TRAIN-EUCLIDEAN_DIST')
train['EUCLIDEAN_DIST'] = train.apply(calc_euc,axis=1)

test['LONGITUDE_DIST'] = test.apply(lambda x: x.LONGITUDE_COUPON - x.LONGITUDE_USER,axis=1)
print('TEST-LONGITUDE_DIST')
test['LATITUDE_DIST'] = test.apply(lambda x: x.LATITUDE_COUPON - x.LATITUDE_USER,axis=1)
print('TEST-LATITUDE_DIST')
test['EUCLIDEAN_DIST'] = test.apply(calc_euc,axis=1)
print('TEST-EUCLIDEAN_DIST')

In [ ]:
x_train = train.drop(['USER_ID_hash', 'COUPON_ID_hash'],axis=1)

x_test = test.drop(['USER_ID_hash', 'COUPON_ID_hash'],axis=1)

In [ ]:
categoricals = x_train.dtypes[x_train.dtypes == 'object'].index.tolist()

In [ ]:
x_train_d = pd.get_dummies(x_train, columns=categoricals)
x_test_d = pd.get_dummies(x_test, columns=categoricals)

x_train_d, x_test_d = x_train_d.align(x_test_d, join='left', axis=1)

x_train_d.fillna(0,inplace=True)
x_test_d.fillna(0,inplace=True)

In [ ]:
x_train_d.set_index('DISPFROM', inplace=True)

In [ ]:
x_test_d.drop(['DISPFROM', 'TARGET'], axis=1, inplace=True)

# Split Data by Week

In [5]:
train_range = []

start = pd.to_datetime('01/08/2012')
end = start
for i in range(23):
    end = end + datetime.timedelta(days=7)

2012-06-17 00:00:00


In [ ]:
xgb_train = x_train_d[(x_train_d.index >= start) & (x_train_d.index < end)]
xgb_eval = x_train_d[(x_train_d.index >= end)]

# Convert to DMatrix

In [ ]:
xgb_train = xgb.DMatrix(xgb_train.drop('TARGET', axis=1), label=xgb_train.TARGET.values.reshape(-1))

In [ ]:
xgb_eval = xgb.DMatrix(xgb_eval.drop('TARGET', axis=1), label=xgb_eval.TARGET.values.reshape(-1))

# Train

In [ ]:
xgb_params = {'booster':'gbtree',
              'objective': 'binary:logistic',
              'eta': 0.001,
              'max_depth': 16,
              'tree_method':'exact',
              'eval_metric': 'logloss'
             }

num_round = 5000

watchlist = [(xgb_train, 'train'), (xgb_eval, 'valid')]
xgb_model = xgb.train(xgb_params, xgb_train, num_round, watchlist, early_stopping_rounds=200, verbose_eval=100)

In [ ]:
xgb_model.save_model('CPP_REPO_xgb_model_23_1.model')

# Predict

In [ ]:
xgb_testA = xgb.DMatrix(x_test_d.iloc[:len(x_test_d)//3])
xgb_testB = xgb.DMatrix(x_test_d.iloc[len(x_test_d)//3:len(x_test_d)//3*2])
xgb_testC = xgb.DMatrix(x_test_d.iloc[len(x_test_d)//3*2:])

In [ ]:
y_predA = xgb_model.predict(xgb_testA)
y_predB = xgb_model.predict(xgb_testB)
y_predC = xgb_model.predict(xgb_testC)

In [ ]:
y_pred = y_predA.tolist() + y_predB.tolist() + y_predC.tolist()

# Convert to Submission Format

In [ ]:
sub = test[['USER_ID_hash','COUPON_ID_hash']]
sub['TARGET'] = y_pred

In [ ]:
grouped = sub.groupby('USER_ID_hash')

In [ ]:
def get_top10(row):
    pred = grouped.get_group(row.USER_ID_hash).sort_values(by=['TARGET'],ascending=False)
    pred = ' '.join(map(str, pred.head(10).COUPON_ID_hash.values))
    return pred

In [ ]:
submission['PURCHASED_COUPONS'] = submission.apply(get_top10, axis=1)

In [ ]:
submission.to_csv('sub_CPP_REPRO_XGB_ONLINE.csv', index=False)

In [ ]:
submission